In [10]:
from scipy.stats import entropy
import nibabel as nib 
import numpy as np 
import matplotlib.pyplot as plt

In [11]:
#check which statistic is most useful to train model with segmentation masks
# statistics include: mean, standard deviation, variance, skewnee, kurtosis, entropy

#load in the data
hard_image = '/valiant02/masi/krishar1/TotalSegmentator_masks_CTkernel_MIDL/B30f_B50f/hard_masked/119178/119178.nii.gz'
soft_image = '/valiant02/masi/krishar1/TotalSegmentator_masks_CTkernel_MIDL/B30f_B50f/soft_masked/119178/119178.nii.gz'
hard_img = nib.load(hard_image)
soft_img = nib.load(soft_image)
hard_data = hard_img.get_fdata()
soft_data = soft_img.get_fdata()

#calculate the statistics
hard_mean = np.mean(hard_data)
soft_mean = np.mean(soft_data)
hard_std = np.std(hard_data)
soft_std = np.std(soft_data)
hard_var = np.var(hard_data)

hard_ent = entropy(hard_data.flatten())
soft_ent = entropy(soft_data.flatten())

In [ ]:
# Load multiple masks. Explore mean and std of the image within the mask 
muscle = nib.load('/valiant02/masi/krishar1/TotalSegmentator_masks_CTkernel_MIDL/B30f_B50f/hard_masked/119178/segmentations/skeletal_muscle.nii.gz')
fat = nib.load('/valiant02/masi/krishar1/TotalSegmentator_masks_CTkernel_MIDL/B30f_B50f/hard_masked/119178/segmentations/subcutaneous_fat.nii.gz')
muscle_data = muscle.get_fdata()
fat_data = fat.get_fdata()

#calculate the statistics using the muscle mask in the image 
muscle_mean = np.mean(hard_data[muscle_data == 1])
muscle_std = np.std(hard_data[muscle_data == 1])
fat_mean = np.mean(hard_data[fat_data == 1])
fat_std = np.std(hard_data[fat_data == 1])

muscle_mean, muscle_std, fat_mean, fat_std

(29.43140775659209, 114.1758257725864, -103.8989985012111, 98.41642368046736)